In [ ]:
import scipy.io as sio
from torch.utils.data import DataLoader 
import torch
from torch import nn
import numpy as np
from torchsummary import summary
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
filename1 = '/content/drive/MyDrive/denoiser/deconvolver_sin1.mat'
#filename2 = '/content/drive/MyDrive/denoiser/data_randomvariance.mat'
batch_size = 64

data1 = sio.loadmat(filename1)
#data2 = sio.loadmat(filename2)

train1 = data1['train']
test1 = data1['test']

#train2 = data2['train']
#test2 = data2['test']

#train = np.concatenate((train1[0:20000,:,:],train2[0:20000,:,:]))?
#test = np.concatenate((test1[0:3000,:,:],test2[0:3000,:,:]))

train_data = DataLoader(train1,batch_size=batch_size,shuffle=True,num_workers=2)
test_data = DataLoader(test1,batch_size=batch_size,shuffle=True,num_workers=2)

In [ ]:
class denoiser(nn.Module):
  def __init__(self):
    super(denoiser,self).__init__()
    self.conv = nn.Sequential(
        
        # nn.Conv1d(1,1,kernel_size=17,padding=8,bias=False),

        # nn.Conv1d(1,1,kernel_size=17,padding=8,bias=False),

        nn.Conv1d(1,2,kernel_size=9,padding=4,bias=False),
        nn.ReLU(inplace=True),

        nn.Conv1d(2,2,kernel_size=17,padding=8,bias=False),
        nn.ReLU(inplace=True),

        # nn.Conv1d(2,2,kernel_size=17,padding=8,bias=False),
        # nn.ReLU(inplace=True),

        # nn.Conv1d(2,2,kernel_size=17,padding=8,bias=False),
        # nn.ReLU(inplace=True),

        # nn.Conv1d(2,2,kernel_size=17,padding=8,bias=False),
        # nn.ReLU(inplace=True),

        nn.Conv1d(2,2,kernel_size=37,padding=18,bias=False),
        nn.ReLU(inplace=True),

        nn.Conv1d(2,1,kernel_size=9,padding=4,bias=False)
    )

  def forward(self,x):
    
    x = self.conv(x)
    return x

In [ ]:
model = denoiser()
weight = sio.loadmat('/content/drive/MyDrive/denoiser/deconv2_pre.mat')
# model._modules['conv'][0]._parameters['weight'].data = torch.from_numpy(weight['conv1'])
# model._modules['conv'][1]._parameters['weight'].data = torch.from_numpy(weight['conv2'])
# model._modules['conv'][2]._parameters['weight'].data = torch.from_numpy(weight['conv3'])
# model._modules['conv'][4]._parameters['weight'].data = torch.from_numpy(weight['conv4'])
# model._modules['conv'][6]._parameters['weight'].data = torch.from_numpy(weight['conv5'])
for i in range(7):
  if i%2==0:
    model._modules['conv'][i]._parameters['weight'].data = torch.from_numpy(weight['conv'+str(i//2+1)])

model._modules['conv'][2]._parameters['weight'].data = torch.rand(2,2,17)
# model._modules['conv'][8]._parameters['weight'].data = torch.rand(2,2,17)
# model._modules['conv'][6]._parameters['weight'].data = torch.rand(2,2,17)
# model._modules['conv'][8]._parameters['weight'].data = torch.rand(2,2,17)
#model._modules['conv'][2]._parameters['weight'].data[:,:,9] = 0.6
#model._modules['conv'][4]._parameters['weight'].data = 0.6*torch.rand(2,8,17)-0.4
#model._modules['conv'][4]._parameters['weight'].data[:,:,9] = 1.5

In [ ]:
for name,value in model.named_parameters():
  if (name=='conv.0.weight')|(name=='conv.4.weight')|(name=='conv.6.weight'):
    value.requires_grad = False

epochs = 700
device = torch.device("cuda:0")
model = model.to(device)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr=1e-3)
#optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=300,gamma=0.1)
loss = nn.MSELoss().to(device)
summary(model, (1, 500))

In [ ]:
train_loss_epoch = []
test_loss_epoch = []
for epoch in range(epochs):
  train_losses = []
  test_losses = []
  for training in train_data:
    model.train()
    x_train = training[:,0,:].view(-1,1,500).type(torch.FloatTensor).to(device)
    y_train = training[:,2,:].view(-1,1,500).type(torch.FloatTensor).to(device)
    optimizer.zero_grad()
    y_pred = model(x_train)
    loss_train = loss(y_pred,y_train)
    loss_train.backward()
    optimizer.step()
    train_losses.append(loss_train.item())
  train_loss_epoch.append(np.mean(train_losses))
  scheduler.step()

  for testing in test_data:
    model.eval()
    x_test = testing[:,0,:].view(-1,1,500).type(torch.FloatTensor).to(device)
    y_test = testing[:,2,:].view(-1,1,500).type(torch.FloatTensor).to(device)
    y_pred = model(x_test)
    loss_test = loss(y_pred,y_test)
    test_losses.append(loss_test.item())
  test_loss_epoch.append(np.mean(test_losses))

  if (epoch%5)==0:
    print("Epoch: %d      train loss: %f      test loss: %f" %(epoch,np.mean(train_losses),np.mean(test_losses)))

  if ((epoch%100)==99)|(epoch==0):
    signal = y_test.view(-1,500).cpu().detach().numpy()
    denoised = y_pred.view(-1,500).cpu().detach().numpy()
    noisy = x_test.view(-1,500).cpu().detach().numpy()
    #SDR = np.mean(np.sum(signal**2,1)/np.sum(denoised**2,1))
    #print("Epoch: %d      SDR: %f" %(epoch,SDR))
    plt.figure(figsize=(10,10))
    plt.subplot(3,1,1)
    plt.plot(signal[0])
    plt.title('Pure Signal')
    plt.subplot(3,1,2)
    plt.plot(denoised[0])
    plt.title('Denoised Signal')
    plt.subplot(3,1,3)
    plt.plot(noisy[0])
    plt.title('Signal with Noise')


In [ ]:
weight_trained = dict()
# weight_trained['conv1'] = model._modules['conv'][0]._parameters['weight'].data.cpu().numpy()
# weight_trained['conv2'] = model._modules['conv'][1]._parameters['weight'].data.cpu().numpy()
# weight_trained['conv3'] = model._modules['conv'][2]._parameters['weight'].data.cpu().numpy()
# weight_trained['conv4'] = model._modules['conv'][4]._parameters['weight'].data.cpu().numpy()
# weight_trained['conv5'] = model._modules['conv'][6]._parameters['weight'].data.cpu().numpy()
for i in range(7):
  if i%2==0:
    weight_trained['conv'+str(i//2+1)] = model._modules['conv'][i]._parameters['weight'].data.cpu().numpy()
sio.savemat('/content/drive/MyDrive/denoiser/deconv2.mat', mdict=weight_trained)